<a href="https://colab.research.google.com/github/seungyeon38/Webtoon_recommendation_system/blob/master/%EC%BD%94%EC%8B%B8%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84%EC%A0%81%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import string
string.punctuation
from math import log
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag
import re
from tqdm import tqdm
from gensim.summarization import keywords


!pip install gensim


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
comic = pd.read_csv('./sample_data/Webtoon Dataset.csv')

In [3]:
# 축약형 
contractions = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", "she's": "she is"}

def regularize_contractions(text):

    contractionfree_list = []
    
    for word in text.split(" "):
      if word in contractions.keys():
        contractionfree_list.append(contractions[word])
      else :
        contractionfree_list.append(word)
      
    return " ".join(contractionfree_list)


In [4]:
#defining the function to remove punctuation
# 글자가 puctuation에 해당하면 " "를, 해당하지 않으면 글자 그대로

def remove_punctuation(text):
    punctuationfree_list = []

    for word in text:
      if word in string.punctuation:
        punctuationfree_list.append(" ")
      else :
        punctuationfree_list.append(word)
    return "".join(punctuationfree_list)


# def remove_punctuation(text):
#     punctuationfree = "".join([i for i in text if i not in string.punctuation])
#     return punctuationfree

In [5]:
# #defining function for tokenization
# 숫자 포함 토큰화
# def tokenization(text):
#     tokens = re.split('\W+', text)

#     return tokens

# 영어만 단어단위로 토큰화
p = re.compile('[a-z]+')

def tokenization(text):
    # print(text)
    result = p.findall(text)
    # print(result)
    return result

In [6]:
#stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')
# print(stopwords[:20]) 

#defining the function to remove stopwords from tokenized text
# stopwords 제거 
def remove_stopwords(text):
    for word in text:
      output= [word for word in text if word not in stopwords]
    return output

In [7]:
#setting lower case
comic['Lower_Summary'] = comic['Summary'].apply(lambda x: x.lower())
comic['Lower_Summary'].head(3)

0    she's young, single and about to achieve her d...
1    after binge-watching beauty videos online, a s...
2    after making a grisly discovery in the country...
Name: Lower_Summary, dtype: object

In [8]:
comic['No_Contraction_Summary'] = comic['Lower_Summary'].apply(lambda x:regularize_contractions(x))
comic['No_Contraction_Summary'].head(3)

0    she is young, single and about to achieve her ...
1    after binge-watching beauty videos online, a s...
2    after making a grisly discovery in the country...
Name: No_Contraction_Summary, dtype: object

In [9]:
comic['Clean_Summary'] = comic['No_Contraction_Summary'].apply(lambda x:remove_punctuation(x))
comic['Clean_Summary'].head(3)

0    she is young  single and about to achieve her ...
1    after binge watching beauty videos online  a s...
2    after making a grisly discovery in the country...
Name: Clean_Summary, dtype: object

In [10]:
#applying function to the column
comic['Tokenied_Summary'] = comic['Clean_Summary'].apply(lambda x: tokenization(x))

comic['Tokenied_Summary'].head(3)

0    [she, is, young, single, and, about, to, achie...
1    [after, binge, watching, beauty, videos, onlin...
2    [after, making, a, grisly, discovery, in, the,...
Name: Tokenied_Summary, dtype: object

In [11]:
#applying the function
comic['No_Stopwords_Summary'] = comic['Tokenied_Summary'].apply(lambda x:remove_stopwords(x))

comic['No_Stopwords_Summary'].head(3)

0    [young, single, achieve, dream, creating, incr...
1    [binge, watching, beauty, videos, online, shy,...
2    [making, grisly, discovery, countryside, small...
Name: No_Stopwords_Summary, dtype: object

In [12]:
# def extract_noun(text):
#   part_of_speech_text = pos_tag(text)
#   result = []
#   for word in part_of_speech_text:
#     if !(word[1] == "NN" or word[1] == "NNS"):
#         result.append(word[0])

#   return result; 

# comic['Noun_Summary'] = comic['No_Stopwords_Summary'].apply(lambda x:extract_noun(x))

# comic['Noun_Summary'].head(3)

In [13]:
stemmer = nltk.stem.SnowballStemmer('english')


def stem(words_list):
  stemmer_words = [stemmer.stem(word) for word in words_list]

  return stemmer_words

comic['Stemmed_Summary'] = comic['No_Stopwords_Summary'].apply(lambda x:stem(x))


comic['Stemmed_Summary'].head(3)

0    [young, singl, achiev, dream, creat, incred, v...
1    [bing, watch, beauti, video, onlin, shi, comic...
2    [make, grisli, discoveri, countrysid, small, t...
Name: Stemmed_Summary, dtype: object

In [14]:
# df1 = comic['No_Stopwords_Summary']
# doc_words_list = df1.values.tolist()

# print(doc_words_list)


def wordlist_to_string(words_list) :
  doc = []
  for doc_word in words_list :
    doc.append(doc_word)
  return " ".join(doc)


comic['Summary_final'] = comic['Stemmed_Summary'].apply(lambda x:wordlist_to_string(x))


comic['Summary_final'].head(3)

0    young singl achiev dream creat incred videogam...
1    bing watch beauti video onlin shi comic book f...
2    make grisli discoveri countrysid small town bo...
Name: Summary_final, dtype: object

In [15]:
def extract_keywords(doc):
  keyword_list = []
  keyword_string = keywords(doc)
  keyword_list = keyword_string.split('\n')

  return keyword_list;


comic['Summary_Keywords'] = comic['Summary_final'].apply(lambda x:extract_keywords(x))


comic.head(3)



,id,Name,Writer,Likes,Genre,Rating,Subscribers,Summary,Update,Reading Link,Lower_Summary,No_Contraction_Summary,Clean_Summary,Tokenied_Summary,No_Stopwords_Summary,Stemmed_Summary,Summary_final,Summary_Keywords
0,0,Let's Play,Leeanne M. Krecic (Mongie),30.6M,Romance,9.62,4.2M,"She's young, single and about to achieve her d...",UP EVERY TUESDAY,https://www.webtoons.com/en/romance/letsplay/l...,"she's young, single and about to achieve her d...","she is young, single and about to achieve her ...",she is young single and about to achieve her ...,"[she, is, young, single, and, about, to, achie...","[young, single, achieve, dream, creating, incr...","[young, singl, achiev, dream, creat, incred, v...",young singl achiev dream creat incred videogam...,"[popular, game, singl, life, critic, stay]"
1,1,True Beauty,Yaongyi,39.9M,Romance,9.60,6.4M,"After binge-watching beauty videos online, a s...",UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/truebeauty...,"after binge-watching beauty videos online, a s...","after binge-watching beauty videos online, a s...",after binge watching beauty videos online a s...,"[after, binge, watching, beauty, videos, onlin...","[binge, watching, beauty, videos, online, shy,...","[bing, watch, beauti, video, onlin, shi, comic...",bing watch beauti video onlin shi comic book f...,"[secret, watch, stand, live, master art]"
2,2,Midnight Poppy Land,Lilydusk,10.4M,Romance,9.81,2.1M,After making a grisly discovery in the country...,UP EVERY SATURDAY,https://www.webtoons.com/en/romance/midnight-p...,after making a grisly discovery in the country...,after making a grisly discovery in the country...,after making a grisly discovery in the country...,"[after, making, a, grisly, discovery, in, the,...","[making, grisly, discovery, countryside, small...","[make, grisli, discoveri, countrysid, small, t...",make grisli discoveri countrysid small town bo...,"[deeper, underworld, intimid, young]"


In [17]:
comic_df = comic[['id','Name', 'Writer', 'Genre', 'Rating',
                 'Summary_Keywords']]
comic_df.head(1)

,id,Name,Writer,Genre,Rating,Summary_Keywords
0,0,Let's Play,Leeanne M. Krecic (Mongie),Romance,9.62,"[popular, game, singl, life, critic, stay]"


In [19]:
from sklearn.feature_extraction.text import CountVectorizer

comic_df['Summary_literal'] = comic_df['Summary_Keywords'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
Summary_mat = count_vect.fit_transform(comic_df['Summary_literal'])
print(Summary_mat.shape)

(569, 3373)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
from sklearn.metrics.pairwise import cosine_similarity

Summary_sim = cosine_similarity(Summary_mat, Summary_mat)
print(Summary_sim.shape)
print(Summary_sim[:2])

(569, 569)
[[1.         0.         0.         ... 0.         0.11396058 0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]]


In [21]:
# 0번 레코드의 경우 94번 레코드와 가장 유사도 높고, 284번 레코드와 가장 유사도 낮음을 나타내는것

Summary_sim_sorted_ind = Summary_sim.argsort()[:, ::-1]
print(Summary_sim_sorted_ind[:1])

[[  0  94 135 443 260 490 417 137 107  39 326 248 319 191 239 118 567 230
   58 533 415   8 250  98 264  18 213 410  37 401 222 471 340  99  15  76
  210  85 355  92  97 418 174 509  55  77  23 188 125 164 211 411  60 354
  182 181 263 237 265 179 178 177 176 180 185 183 184 186 187 189 190 262
  261 192 193 194 175 173 266 196 154 155 273 272 156 157 271 158 270 269
  159 160 161 162 163 268 165 267 166 167 168 169 170 171 172 195 259 197
  243 217 218 219 220 221 245 223 224 225 226 244 242 215 227 241 228 229
  240 231 232 238 233 234 235 216 214 236 204 258 257 198 199 256 200 255
  254 201 202 203 253 246 205 206 207 208 209 252 152 251 212 249 247 153
  140 151  36  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
   56  57  59  61  62  63  64  65  66  67  68  69  70  71  38  35 150  34
    1   2   3   4   5   6   7   9  10  11  12  13  14  16  17  19  20  21
   22  24  25  26  27  28  29  30  31  32  33  72  73  74  75 117 119 120
  121 122 123 124 126 127 128 129 130 

In [22]:
def find_sim_comic(df, sorted_ind, c_name, top_n=10):
    comic_name = df[df['Name'] == c_name]
    comic_index = comic_name.index.values
    similar_indexes = sorted_ind[comic_index, :(top_n)]
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]

In [24]:
# 'Death of a Pop Star'와 비슷한 키워드 가진 웹툰 10개 추천
similar_comic = find_sim_comic(comic_df, Summary_sim_sorted_ind, 'Death of a Pop Star',10)
similar_comic[['Name', 'Rating' , 'Summary_Keywords']]

[[ 30 236 441 101 277 174 414 181 182 180]]


,Name,Rating,Summary_Keywords
30,Death of a Pop Star,9.34,"[lucki, reaper, dead]"
236,Neolithic Girl,9.38,"[live, save, clan, death, immort, dead]"
441,unOrdinary,9.76,"[john, destini, school social, frenemi dead]"
101,Gourmet Hound,9.82,"[tast, love year lucki accid, woman]"
277,Rise from Ashes,9.70,"[world, dead, ghost midst, great, abandon hous]"
174,Version Day and Night,9.53,"[shun, privat, doubl life, shake, dagger lucki]"
414,TACIT,9.65,"[young rubi, father, dead zombi, futur world, ..."
181,My Gently Raised Beast,9.85,"[amon, blondina live, like creatur, half, rift..."
182,Castle Swimmer,9.84,"[futur, sea, beacon lead, peopl, born, die]"
180,Cursed Princess Club,9.80,"[princess, gwendolyn live, club, accident, mol..."


In [25]:
# 평점 순 나열

comic_df[['Name','Rating', 'Summary_Keywords']].sort_values('Rating',ascending=False)[:10]

,Name,Rating,Summary_Keywords
195,Eleceed,9.93,"[kayden, world, cat secret, jiwoo, evil, littl..."
93,Your Letter,9.93,"[sori, new, middl school, person, trail letter..."
130,Spirit Fingers,9.91,"[ami, art, awkward, self, studi]"
24,Purple Hyacinth,9.91,"[detect, inabl, offic, team, gone, defenseless]"
187,Omniscient Reader,9.90,"[novel, dokja, world, surviv, favorit]"
191,See You in My 19th Life,9.89,"[life, past live, ban]"
413,Wind Breaker,9.89,"[student, outsid, new, crew]"
88,Bastard,9.87,[]
7,Teenage Mercenary,9.87,"[surviv, ijin, school, year, new tactic maneuv..."
161,The Remarried Empress,9.87,"[empress perfect, navier, husband, divorc, soc..."
